In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q langchain langchain_community sentence_transformers modelscope torch datasets peft trl bitsandbytes accelerate transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.5 MB/s eta 

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
import pandas as pd
from langchain.schema import Document
import torch
from sentence_transformers import SentenceTransformer
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def split_dialogue_by_empty_lines(conversation):
    dialogue_units = conversation.strip().split('\n\n')
    return dialogue_units

In [ ]:
file_path = '/content/drive/MyDrive/DATA298B/data/interview_transcript_Q&A.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    texts = file.read()

split_dialogues = split_dialogue_by_empty_lines(texts)

for idx, dialogue in enumerate(split_dialogues, 1):
    print(f"Dialogue {idx}:")
    print(dialogue)
    print("-----")

Dialogue 1:
Q: This is a tape-recorded interview with Professor Richard Feynman, in his home in Altadena, California, on March 4, 1966. Charles Weiner will occasionally interrupt with questions. Let’s start at the beginning.
A: You start by interrupting me with a question.
-----
Dialogue 2:
Q: That’s right. It saves interruption later. Let’s talk about the earliest recollections you have of your family, your neighborhood and your home.
A: Nothing scientific, just a simple —? I don’t know what to say. That’s a hard one. I have lots of recollections, little bits and pieces.
-----
Dialogue 3:
Q: Let me ask where you lived in Far Rockaway, was this —?
A: I was born somewhere in New York, but I don’t remember anything except Far Rockaway, and then we went for some time, I remember, to a house in a place called Baldwin, on Long Island, where I think we lived a year or a half a year or a summer. And then, sometime after Far Rockaway, we went to live in Cedarhurst, but then by that time I was 

In [ ]:
documents = [Document(page_content=dialogue) for dialogue in split_dialogues]
print(documents)

[Document(metadata={}, page_content='Q: This is a tape-recorded interview with Professor Richard Feynman, in his home in Altadena, California, on March 4, 1966. Charles Weiner will occasionally interrupt with questions. Let’s start at the beginning.\nA: You start by interrupting me with a question.'), Document(metadata={}, page_content='Q: That’s right. It saves interruption later. Let’s talk about the earliest recollections you have of your family, your neighborhood and your home.\nA: Nothing scientific, just a simple —? I don’t know what to say. That’s a hard one. I have lots of recollections, little bits and pieces.'), Document(metadata={}, page_content='Q: Let me ask where you lived in Far Rockaway, was this —?\nA: I was born somewhere in New York, but I don’t remember anything except Far Rockaway, and then we went for some time, I remember, to a house in a place called Baldwin, on Long Island, where I think we lived a year or a half a year or a summer. And then, sometime after Far

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name = "moka-ai/m3e-base"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings.client = SentenceTransformer(model_name, device=device)

<ipython-input-9-f5eeaa36f5a7>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/932 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
feynman_db = Chroma.from_documents(documents=documents, embedding=embeddings,persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman")
feynman_db.persist()

<ipython-input-10-9ff194367e0a>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  feynman_db.persist()


In [ ]:
db = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={'k': 10})


<ipython-input-11-5edd7a614840>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(persist_directory="/content/drive/MyDrive/DATA298B/ChormaDBs/feynman", embedding_function=embeddings)


In [ ]:
docs = retriever.invoke('''Professor Feynman, how is going?''')
print(docs)

[Document(metadata={}, page_content='Q: This is a tape-recorded interview with Professor Richard Feynman, in his home in Altadena, California, on March 4, 1966. Charles Weiner will occasionally interrupt with questions. Let’s start at the beginning.\nA: You start by interrupting me with a question.'), Document(metadata={}, page_content='Q: What is her name?\nA: Joan Feynman — Joan Hirschberg.'), Document(metadata={}, page_content='Q: Did he follow through on the technical or scientific education?\nA: No. I think it was family influences, the importance of money and business, so that be became more a salesman than an honest scientific man. A mixture. At first he had developed a process for metal-plating plastics, and done some research to develop it. Later, I got a job with him, from 1939, probably.'), Document(metadata={}, page_content='Q: Where did she did get her Ph.D.?\nA: She got it at Syracuse University. She got her BS at Oberlin, Ohio. Now she’s working for Ames Research in — sp